In [57]:

#spark operation library...
from pyspark.sql.functions import *
from pyspark.sql.types import *

#python important library...
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


#Ml model and evaluation technique...

from pyspark.ml.feature import VectorAssembler,StringIndexer,OneHotEncoder,VectorSlicer,VectorIndexer
from pyspark.ml.classification import LogisticRegression,RandomForestClassifier,DecisionTreeClassifier
from pyspark.ml.linalg import Vectors
from pyspark.ml import pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator

from sklearn.metrics import roc_curve,auc,log_loss
from pyspark.sql import window,WindowSpec

In [2]:
train_df = spark.read.csv('homesite-quote-conversion/train.csv',header=True)

In [ ]:
len(train_df.columns)

In [12]:
train_df.limit(10).toPandas()

QuoteNumber Original_Quote_Date QuoteConversion_Flag Field6 Field7  Field8  \
0           1          2013-08-16                    0      B     23  0.9403   
1           2          2014-04-22                    0      F      7  1.0006   
2           4          2014-08-25                    0      F      7  1.0006   
3           6          2013-04-15                    0      J     10  0.9769   
4           8          2014-01-25                    0      E     23  0.9472   
5          12          2014-01-18                    0      E     14  0.9472   
6          13          2013-11-01                    0      J     23  0.9258   
7          14          2014-05-14                    0      B     25  0.9153   
8          18          2013-06-19                    0      J     10  0.9691   
9          19          2013-06-18                    0      F     11  0.9919   

   Field9 Field10 Field11 Field12  ... GeographicField59A GeographicField59B  \
0  0.0006     965    1.02       N  ...                  9                  9   
1   0.004     548  1.2433       N  ...                 10                 10   
2   0.004     548  1.2433       N  ...                 15                 18   
3  0.0004   1,165  1.2665       N  ...                  6                  5   
4  0.0006   1,487  1.3045       N  ...                 18                 22   
5  0.0006   1,487  1.3045       N  ...                 10                 11   
6  0.0004   1,165  1.2665       N  ...                 18                 22   
7  0.0007     935    1.02       N  ...                  6                  5   
8  0.0004   1,165  1.2665       N  ...                  2                  1   
9  0.0038     564  1.1886       N  ...                 10                 11   

  GeographicField60A GeographicField60B GeographicField61A GeographicField61B  \
0                 -1                  8                 -1                 18   
1                 -1                 11                 -1                 17   
2                 -1                 21                 -1                 11   
3                 -1                 10                 -1                  9   
4                 -1                 10                 -1                 11   
5                 -1                 23                 -1                 25   
6                 -1                 19                 -1                 13   
7                 -1                  3                 -1                 12   
8                 -1                  3                 -1                  2   
9                 -1                  9                 -1                 22   

  GeographicField62A GeographicField62B GeographicField63 GeographicField64  
0                 -1                 10                 N                CA  
1                 -1                 20                 N                NJ  
2                 -1                  8                 N                NJ  
3                 -1                 21                 N                TX  
4                 -1                 12                 N                IL  
5                 -1                 14                 N                IL  
6                 -1                  8                 N                TX  
7                 -1                 13                 N                CA  
8                 -1                 13                 N                TX  
9                 -1                 18                 N                NJ  

[10 rows x 299 columns]

In [15]:
train_df = train_df.withColumnRenamed('QuoteConversion_Flag','label')
train_df = train_df.withColumnRenamed('QuoteNumber','id')

In [16]:
train_df.limit(10).toPandas()

id Original_Quote_Date label Field6 Field7  Field8  Field9 Field10 Field11  \
0   1          2013-08-16     0      B     23  0.9403  0.0006     965    1.02   
1   2          2014-04-22     0      F      7  1.0006   0.004     548  1.2433   
2   4          2014-08-25     0      F      7  1.0006   0.004     548  1.2433   
3   6          2013-04-15     0      J     10  0.9769  0.0004   1,165  1.2665   
4   8          2014-01-25     0      E     23  0.9472  0.0006   1,487  1.3045   
5  12          2014-01-18     0      E     14  0.9472  0.0006   1,487  1.3045   
6  13          2013-11-01     0      J     23  0.9258  0.0004   1,165  1.2665   
7  14          2014-05-14     0      B     25  0.9153  0.0007     935    1.02   
8  18          2013-06-19     0      J     10  0.9691  0.0004   1,165  1.2665   
9  19          2013-06-18     0      F     11  0.9919  0.0038     564  1.1886   

  Field12  ... GeographicField59A GeographicField59B GeographicField60A  \
0       N  ...                  9                  9                 -1   
1       N  ...                 10                 10                 -1   
2       N  ...                 15                 18                 -1   
3       N  ...                  6                  5                 -1   
4       N  ...                 18                 22                 -1   
5       N  ...                 10                 11                 -1   
6       N  ...                 18                 22                 -1   
7       N  ...                  6                  5                 -1   
8       N  ...                  2                  1                 -1   
9       N  ...                 10                 11                 -1   

  GeographicField60B GeographicField61A GeographicField61B GeographicField62A  \
0                  8                 -1                 18                 -1   
1                 11                 -1                 17                 -1   
2                 21                 -1                 11                 -1   
3                 10                 -1                  9                 -1   
4                 10                 -1                 11                 -1   
5                 23                 -1                 25                 -1   
6                 19                 -1                 13                 -1   
7                  3                 -1                 12                 -1   
8                  3                 -1                  2                 -1   
9                  9                 -1                 22                 -1   

  GeographicField62B GeographicField63 GeographicField64  
0                 10                 N                CA  
1                 20                 N                NJ  
2                  8                 N                NJ  
3                 21                 N                TX  
4                 12                 N                IL  
5                 14                 N                IL  
6                  8                 N                TX  
7                 13                 N                CA  
8                 13                 N                TX  
9                 18                 N                NJ  

[10 rows x 299 columns]

In [62]:
# target_percentage = train_df.groupBy('label').count().sort(col("count").desc())\
#                             .withColumn('total',sum(col('count')).over(WindowSpec))\
#                             .withColumn('Percent',col("count")*100/col('total'))

In [68]:
cat_cols = [item[0] for item in train_df.dtypes if item[1].startswith('string')] 
print("cat_cols:", len(cat_cols))

cat_cols: 299


In [72]:
num_cols = [item[0] for item in train_df.dtypes if item[1].startswith('int') | item[1].startswith('double')] 
print("num_cols:", num_cols)

num_cols: []
